# Exercise: Find top 10 movies 

Dataset: movielens.org database https://github.com/abulbasar/data/tree/master/ml-latest-small

In dataset each movies (movies.csv) contain 1 to many ratings (ratings.csv)

1. Load movies.csv in movies dataframe
2. Load ratings.csv in ratings dataframe
3. Find how many ratings "Toy Story (1995)" have got
4. Find avg rating and number of rating for each movieid
5. Filter out all movies that have got fewer than 100 ratings
6. Join movies and ratings dataframe to show title, avg_rating 
7. Sort the data based on avg rating  desc order
8. Select top 10 movies.

    
